In [1]:
import random
import tensorflow as tf
import numpy as np
from copy import copy
import os
from scipy.misc import toimage
from scipy import signal
from scipy.misc import imread, imsave, imresize
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
# imshow(image)
# plt.show()
# toimage(image).show()

tf.set_random_seed(777)

In [2]:
gpu_config = tf.ConfigProto(device_count={'GPU':1})  # only use GPU no.1 change this!
gpu_config.gpu_options.allow_growth = True # only use required resource(memory)
gpu_config.gpu_options.per_process_gpu_memory_fraction = 0.5 # restrict to 50%

In [3]:

def search(dirname):
    filenames = os.listdir(dirname)
    full_filenames = []
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        full_filenames.append(full_filename)
        #print (full_filename)
    return full_filenames
"""
def Image_search(Image_dir,num_image):
    images=[]
    for i in range(num_image):
        image_raw = imread(Image_dir[i])
        image = imresize(image_raw,(300,300))
        images.append(image)
    return images
"""

def Image_search(Image_dir,num_image):
    images=np.zeros([1,200*200*3])
    for i in range(num_image):
        image_raw = imread(Image_dir[i])
        image = imresize(image_raw,(200,200))
        image = np.reshape(image,(1,200*200*3))/255
        image = image # - np.mean(image)
        images = np.append(images,image,axis=0)
    return images[1:]

def label2onehot(label,num_class):
    one_hot = np.zeros((len(label),num_class))
    for idx,l in enumerate(label):
        one_hot[idx,l] = 1
    return one_hot

search("/home/junhyun/Data/Dataset")

Total_images = np.zeros([1,200*200*3])
Total_labels = np.zeros([1])

Image_dir = search("/home/junhyun/Data/Dataset/Airplanes")
airplanes_images = Image_search(Image_dir,350)
airplanes_labels = np.ones(len(airplanes_images))*0

Total_images = np.append(Total_images,airplanes_images,axis=0)
Total_labels = np.append(Total_labels,airplanes_labels,axis=0)


Image_dir = search("/home/junhyun/Data/Dataset/Binocular")
binocular_images = Image_search(Image_dir,len(Image_dir))
binocular_labels = np.ones(len(binocular_images))*1

Total_images = np.append(Total_images,binocular_images,axis=0)
Total_labels = np.append(Total_labels,binocular_labels,axis=0)

Image_dir = search("/home/junhyun/Data/Dataset/grapes")
grapes_images = Image_search(Image_dir,len(Image_dir))
grapes_labels = np.ones(len(grapes_images))*2

Total_images = np.append(Total_images,grapes_images,axis=0)
Total_labels = np.append(Total_labels,grapes_labels,axis=0)

Image_dir = search("/home/junhyun/Data/Dataset/Leopards")
leopards_images = Image_search(Image_dir,len(Image_dir))
leopards_labels = np.ones(len(leopards_images))*3

Total_images = np.append(Total_images,leopards_images,axis=0)
Total_labels = np.append(Total_labels,leopards_labels,axis=0)

Image_dir = search("/home/junhyun/Data/Dataset/Motorbikes")
motorbikes_images = Image_search(Image_dir,350)
motorbikes_labels = np.ones(len(motorbikes_images))*4

Total_images = np.append(Total_images,motorbikes_images,axis=0)
Total_labels = np.append(Total_labels,motorbikes_labels,axis=0)

Image_dir = search("/home/junhyun/Data/Dataset/watch")
watch_images = Image_search(Image_dir,len(Image_dir))
watch_labels = np.ones(len(watch_images))*5

Total_images = np.append(Total_images,watch_images,axis=0)
Total_labels = np.append(Total_labels,watch_labels,axis=0)

Image_dir = search("/home/junhyun/Data/Dataset/Faces_easy")
faces_easy_images = Image_search(Image_dir,350)
faces_easy_labels = np.ones(len(faces_easy_images))*6

Total_images = np.append(Total_images,faces_easy_images,axis=0)[1:]
Total_labels = np.append(Total_labels,faces_easy_labels,axis=0)[1:]

In [4]:
Total_one_hot = Total_labels.astype(int)
Total_one_hot = label2onehot(np.transpose(Total_one_hot),7)

In [5]:
def shuffle_img_label(img,label):
    tmp_img = np.zeros_like(img)
    tmp_label = np.zeros_like(label)
    Order = list(range(0,len(label)))
    np.random.shuffle(Order)
    for idx,order in enumerate(Order):
        tmp_img[idx] = img[order]
        tmp_label[idx] = label[order]
    return tmp_img, tmp_label

Shuffle_image, Shuffle_label = shuffle_img_label(Total_images,Total_one_hot)
# Shuffle_image, Shuffle_label = shuffle_img_label(Total_images,Total_one_hot)

In [6]:
def RGB_flip_LeftRight(Img):
    
    Img_clone = copy(Img)
    Flip_Img_result = np.zeros_like(Img_clone)

    for i in range(Img_clone.shape[0]):
        Flip_Img = np.reshape(Img_clone[i,:],[200,200,3])
    
        for k in range(3):
            for j in range(Flip_Img.shape[0]):            
                Flip_Img[j,:,k] = Flip_Img[j,::-1,k]
    
    Flip_Img = np.reshape(Flip_Img,[1,200*200*3])
    Flip_Img_result[i,:] = Flip_Img

    return Flip_Img_result

In [7]:
Train_images = Shuffle_image[0:1500]
Train_one_hot = Shuffle_label[0:1500]

Flip_images = RGB_flip_LeftRight(Train_images)

Train_images = np.append(Train_images,Flip_images,axis=0)
Train_one_hot = np.append(Train_one_hot,Train_one_hot,axis=0)


Test_images = Shuffle_image[1500:]
Test_one_hot = Shuffle_label[1500:]

In [8]:
class Model:
    def __init__(self, sess, name):
        self.sess=sess
        self.name=name
        self._build_net()
        
    def _build_net(self):
        self.training= tf.placeholder(tf.bool)
        self.X = tf.placeholder(tf.float32,[None, 200*200*3])
        X_img = tf.reshape(self.X, [-1,200,200,3])
        self.Y = tf.placeholder(tf.float32,[None,7])
        
        
        
        self.W1 = tf.Variable(tf.random_normal([3, 3, 3, 64], stddev=0.01))
        
        L1 = tf.nn.conv2d(X_img, self.W1, strides=[1, 1, 1, 1], padding='SAME')
#         L1 = tf.nn.relu(L1)
        
        
#         self.conv_W = tf.layers.conv2d(inputs = X_img, \
#                                        filters = 64, \
#                                        kernel_size=[3, 3], \
#                                        padding = "SAME",\
#                                        activation=tf.nn.relu,\
#                                        name = "conv_W", \
#                                        kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
        
        conv1 = tf.layers.conv2d(inputs = L1, filters = 64, kernel_size=[3, 3], padding = "SAME",\
                                 activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
        conv12 = tf.layers.conv2d(inputs = conv1, filters = 64, kernel_size=[3, 3], padding = "SAME",\
                                  activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())

        pool1 = tf.layers.max_pooling2d(inputs=conv12, pool_size = [2, 2], padding="SAME", strides=2)
#         drop1 = tf.layers.dropout(inputs=pool1, rate=0.7, training = self.training)
        
#         conv2 = tf.layers.conv2d(inputs = pool1, filters = 128, kernel_size=[3, 3], padding = "SAME", \
#                                  activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#         conv22 = tf.layers.conv2d(inputs = conv2, filters = 128, kernel_size=[3, 3], padding = "SAME", \
#                                   activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#         conv23 = tf.layers.conv2d(inputs = conv2, filters = 128, kernel_size=[3, 3], padding = "SAME", \
#                                   activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#         pool2 = tf.layers.max_pooling2d(inputs=conv23, pool_size = [2, 2], padding="SAME", strides=2)
#         drop2 = tf.layers.dropout(inputs=pool2, rate=0.7, training = self.training)
        
        conv3 = tf.layers.conv2d(inputs = pool1, filters = 128, kernel_size=[3, 3], padding = "SAME", \
                                 activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
        conv32 = tf.layers.conv2d(inputs = conv3, filters = 128, kernel_size=[3, 3], padding = "SAME", \
                                  activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
        conv33 = tf.layers.conv2d(inputs = conv3, filters = 128, kernel_size=[3, 3], padding = "SAME", \
                                  activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#         pool3 = tf.layers.max_pooling2d(inputs=conv33, pool_size = [2, 2], padding="SAME", strides=2)
#         drop3 = tf.layers.dropout(inputs=pool3, rate=0.7, training = self.training)
        
        conv4 = tf.layers.conv2d(inputs = conv33, filters = 128, kernel_size=[3, 3], padding = "SAME", \
                                 activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
        conv42 = tf.layers.conv2d(inputs = conv4, filters = 256, kernel_size=[3, 3], padding = "SAME", \
                                  activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#         conv43 = tf.layers.conv2d(inputs = conv4, filters = 128, kernel_size=[3, 3], padding = "SAME", \
#                                   activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
        pool4 = tf.layers.max_pooling2d(inputs=conv42, pool_size = [2, 2], padding="SAME", strides=2)
#         drop4 = tf.layers.dropout(inputs=pool4, rate=0.7, training = self.training)
        
        conv5 = tf.layers.conv2d(inputs = pool4, filters = 256, kernel_size=[3, 3], padding = "SAME", \
                                 activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
        conv52 = tf.layers.conv2d(inputs = conv5, filters = 256, kernel_size=[3, 3], padding = "SAME", \
                                  activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#         conv53 = tf.layers.conv2d(inputs = conv5, filters = 256, kernel_size=[3, 3], padding = "SAME", \
#                                   activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())        
        pool5 = tf.layers.max_pooling2d(inputs=conv52, pool_size = [2, 2], padding="SAME", strides=2)
#         drop5 = tf.layers.dropout(inputs=pool5, rate=0.7, training = self.training)
             
        conv6 = tf.layers.conv2d(inputs = pool5, filters = 128, kernel_size=[3, 3], padding = "SAME", \
                                 activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
#         conv62 = tf.layers.conv2d(inputs = conv6, filters = 512, kernel_size=[3, 3], padding = "SAME", activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
#         conv63 = tf.layers.conv2d(inputs = conv62, filters = 512, kernel_size=[3, 3], padding = "SAME", activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))        
        pool6 = tf.layers.max_pooling2d(inputs=conv6, pool_size = [2, 2], padding="SAME", strides=2)
#         drop6 = tf.layers.dropout(inputs=pool6, rate=0.7, training = self.training)
             
        
        flat = tf.reshape(pool6,[-1,13*13*128])
        dense4 = tf.layers.dense(inputs=flat,units=4096,activation=tf.nn.relu,\
                                 kernel_initializer = tf.contrib.layers.xavier_initializer(),\
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001))
        drop4 = tf.layers.dropout(inputs=dense4,rate=0.4,training = self.training)
        batch4 = tf.layers.batch_normalization(drop4,training= True)
        
        dense5 = tf.layers.dense(inputs=batch4,units=4096,activation=tf.nn.relu,\
                                 kernel_initializer = tf.contrib.layers.xavier_initializer(),\
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001))
        drop5 = tf.layers.dropout(inputs=dense5,rate=0.4,training = self.training)
        batch5 = tf.layers.batch_normalization(drop5,training= True)


        dense6 = tf.layers.dense(inputs=batch5,units=1000,activation=tf.nn.relu,\
                                 kernel_initializer = tf.contrib.layers.xavier_initializer(),\
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001))
        drop6 = tf.layers.dropout(inputs=dense6,rate=0.4,training = self.training)
        batch6 = tf.layers.batch_normalization(drop6,training= True)
        
        self.logits = tf.layers.dense(inputs=batch6,units=7,\
                                      kernel_initializer = tf.contrib.layers.xavier_initializer(),\
                                      kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001))
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        self.prediction = tf.argmax(self.logits,1)
        self.correct_prediction = tf.equal(self.prediction,tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction,tf.float32))
        
        self.gr = tf.get_default_graph()
        
    def predict(self,x_test,training=False):
        return self.sess.run(self.logits,feed_dict={self.X:x_test, self.training:training})
        
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test, self.Y:y_test, self.training:training})
        
    def train(self,x_test,y_test,training=True):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_test,self.Y:y_test, self.training:training})
    
    def get_weight(self):
        Weight = sess.run(self.W1)
        return Weight
    

In [9]:
epochs=500
batch_size=10
learning_rate=0.005

In [10]:
sess = tf.Session(config=gpu_config)

models=[]
num_models=1
for m in range(num_models):
    models.append(Model(sess,"model"+str(m)))

sess.run(tf.global_variables_initializer())


In [11]:
print("Learning start!!")

for epoch in range(epochs):
    learning_rate = learning_rate*0.995
    avg_cost_list = np.zeros(len(models))
    total_batch = int(len(Train_one_hot)/batch_size)
    for i in range(total_batch):
        batch_xs = Train_images[i*batch_size:(i+1)*batch_size]
        batch_ys = Train_one_hot[i*batch_size:(i+1)*batch_size]
        
        for m_idx,m in enumerate(models):
            c,_ = m.train(batch_xs,batch_ys)
            avg_cost_list[m_idx] += c/total_batch
    print("epoch : ", "%04d"%(epoch+1), " cost : ", avg_cost_list)
    
print("Learning finished!!")

Learning start!!


KeyboardInterrupt: 

In [12]:

test_batch_size =4

total_batch = int(Test_images.shape[0]/test_batch_size)
acc = 0
for m_idx,m in enumerate(models):

    for i in range(total_batch):
        batch_xs = Test_images[i*test_batch_size:(i+1)*test_batch_size]
        batch_ys = Test_one_hot[i*test_batch_size:(i+1)*test_batch_size]
        acc += m.get_accuracy(batch_xs,batch_ys)

    print(m_idx, "Accuracy = ", float(acc)/float(total_batch))
    
    

# test_size = len(Test_one_hot)
# prediction = np.zeros(test_size*7).reshape(test_size,7)

# for m_idx,m in enumerate(models):
#     print(m_idx, "Accuracy = ", m.get_accuracy(Test_images,Test_one_hot))
#     p = m.predict(Test_images)
#     prediction +=p

# ensemble_correct = tf.equal(tf.argmax(prediction,1),tf.argmax(Test_one_hot,1))
# ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct,tf.float32))

# print("Ensemble Accuracy = ", sess.run(ensemble_accuracy) )

(0, 'Accuracy = ', 0.44140625)


In [19]:
models[0].predict(Test_images)

array([[  7.79233813e-01,  -2.07577419e+00,  -1.21688664e+00,
          2.02715278e+00,  -5.10181379e+00,   1.85337067e-01,
          2.86026073e+00],
       [ -1.07026768e+01,   3.24990845e+00,   4.11540604e+00,
          2.83889604e+00,  -5.89954567e+00,   7.04104805e+00,
          1.15791607e+00],
       [  1.70338428e+00,  -1.26990891e+00,   2.83647752e+00,
         -2.50867677e+00,   1.12539892e+01,  -6.80820704e+00,
         -1.52159166e+00],
       [  5.92615747e+00,  -5.09203434e+00,  -2.85745239e+00,
          5.29753745e-01,  -7.50579214e+00,   3.79447389e+00,
          3.34223294e+00],
       [  7.21456170e-01,  -5.07615566e+00,  -1.89143491e+00,
         -9.68863010e+00,   1.76491165e+01,  -3.97123766e+00,
         -1.30406737e+00],
       [ -4.03597498e+00,  -2.32412434e+00,   6.61122739e-01,
         -1.55065084e+00,  -9.07968235e+00,   1.75308590e+01,
         -4.34001398e+00],
       [  7.79233813e-01,  -2.07577419e+00,  -1.21688664e+00,
          2.02715278e+00,  -5.10

In [38]:
convWeight = sess.run(tf.transpose(models[0].get_weight()))
Origin_data = Test_image[15][0:1024]
Weight = convWeight[3][0]
32*32

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [ ]:
Weight.shape

In [ ]:
Origin_data.shape

In [ ]:
Origin_image = np.reshape(Origin_data,[32,32])

In [ ]:
import scipy.misc

i_width = 300
i_height = 300

Big_image = scipy.misc.imresize(Origin_image, (i_height, i_width))
print(Big_image.shape)
Origin_image = Big_image

In [ ]:
toimage(Origin_image).show()

In [ ]:
Conv_image = signal.convolve2d(Origin_image,Weight,'same')

Conv_image -= Conv_image.min()

In [ ]:
toimage(Conv_image).show()

In [ ]:
print(Origin_image.max())
print(Origin_image.min())

print(Conv_image.max())
print(Conv_image.min())


In [ ]:
Edge_image = Origin_image-3*Conv_image

In [ ]:
print(Edge_image.max())
print(Edge_image.min())
Edge_image -= Edge_image.min()
Edge_image = Edge_image/Edge_image.max()
print(Edge_image.max())
print(Edge_image.min())

In [ ]:
toimage(Edge_image).show()

In [ ]:
toimage(Weight).show()